#### ***Import our Libraries Here***

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

#### ***EXTRACTION***

In [ ]:
df = pd.read_csv('data/crypto-markets.csv')
df.head(3)

###### **Get Information on dataframe / Overview of the dataframe**

In [ ]:
df.info()

#### ***TRANSFORMATION***
##### **Data Cleaning**

Remove All Null Values from dataframe if any

In [ ]:
df.dropna(inplace=True)

###### Since we only need the asset, name, date, open, high, low and close field; we can drop the unimportant columns

In [ ]:
df.drop(labels=['slug', 'ranknow', 'volume', 'market', 'close_ratio', 'spread'], inplace=True, axis=1)

###### Currently we need only the Bitcoin, Ripple and Litecoin fields

In [ ]:
assetsCodes = ['BTC','XRP','LTC']

In [ ]:
df.drop(pd.Index(np.where(df['asset'].isin(assetsCodes)==False)[0]), inplace = True)

###### Our final approach to cleaning the data is to reset the index

In [ ]:
df.reset_index(drop=True, inplace=True)
df.head(2)

#### ***LOADING***
##### **Loading the dataframe into SQLite - This will act as our datawarehouse**

##### SQLite

In [ ]:
conn = sqlite3.connect('session.db')
print(conn)

In [ ]:
# Drop a table name Crypto id it exists already
try:
    conn.execute('DROP TABLE IF EXISTS `Cryptocurrency` ')
except Exception as e:
    print(str(e))

In [ ]:
# Create a new Table name as Cryptocurrency
try:
    conn.execute('''
        CREATE TABLE Cryptocurrency
        (ID     INTEGER PRIMARY KEY,
         ASSET  TEXT   NOT NULL,
         NAME  TEXT  NOT NULL,
         Date   datetime,
         Open   Float DEFAULT 0,
         High   Float DEFAULT 0,
         Low    Float DEFAULT 0,
         Close  Float DEFAULT 0);
    ''')
    print ("Table created successfully");
except Exception as e:
    print(str(e))
    print('Table Creation Failed')
finally:
    conn.close()

In [ ]:
df_list = df.values.tolist()

# lets make new connection to Insert crypto data in SQL DB
conn = sqlite3.connect('session.db')

# make a cursor - it will help with querying SQL DB
cur = conn.cursor()

try:
    cur.executemany("INSERT INTO Cryptocurrency(ASSET, NAME, Date, Open, High, Low, Close) VALUES (?,?,?,?,?,?,?)", df_list)
    conn.commit()
    print('Data Inserted Successfully')
except Exception as e:
    print(str(e))
    print('Data Insertion Failed')
finally:
    # finally block will help with always closing the connection to DB even in case of error.
    conn.close()

In [ ]:
# Let's Read data from DB to verify it

conn = sqlite3.connect('session.db')
rows = conn.cursor().execute('Select * from Cryptocurrency')
# print(rows[:2])
for row in rows:
    print(row)
conn.close()